# İş Problemi
FLO satış ve pazarlama faaliyetleri için roadmap belirlemek istemektedir. Şirketin orta uzun vadeli plan yapabilmesi için var olan müşterilerin gelecekte şirkete sağlayacakları potansiyel değerin tahmin edilmesi gerekmektedir.

İlk olarak kütüphanelerimizi ekleyelim ve output ayarlarımızı yapalım.

In [2]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler


pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

## Görev 1: Veriyi Hazırlama

### Adım 1: flo_data_20K.csv verisini okuyunuz.

In [8]:
df_ = pd.read_csv("Datasets/flo_data_20k.csv")
df = df_.copy()

### Adım 2: Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds fonksiyonlarını tanımlayınız.
Not: cltv hesaplanırken frequency değerleri integer olması gerekmektedir.Bu nedenle alt ve üst limitlerini round() ile yuvarlayınız

In [9]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit,up_limit

In [10]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit, 0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit, 0)

### Adım 3: "order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline", "customer_value_total_ever_online" değişkenlerinin aykırı değerleri varsa baskılayanız.

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000,3.111,4.226,1.000,1.000,2.000,4.000,200.000
order_num_total_ever_offline,19945.000,1.914,2.063,1.000,1.000,1.000,2.000,109.000
customer_value_total_ever_offline,19945.000,253.923,301.533,10.000,99.990,179.980,319.970,18119.140
customer_value_total_ever_online,19945.000,497.322,832.602,12.990,149.980,286.460,578.440,45220.130


Tüm sütunlarda max değer gerekenden yüksek olduğu için aykırı değerleri baskılamak için gerekli olan fonksiyonu uyguluyorum.


In [12]:
replace_with_thresholds(df, "order_num_total_ever_online")
replace_with_thresholds(df, "order_num_total_ever_offline")
replace_with_thresholds(df, "customer_value_total_ever_offline")
replace_with_thresholds(df,"customer_value_total_ever_online")

### Adım 4: Omnichannel müşterilerin hem online'dan hem de offline platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.

In [13]:
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

Offline ve Online olarak ayrı ayrı analiz yapmamız istenmediği için bu değerleri birleştirmek daha mantıklı olacak.

### Adım 5: Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.

In [14]:
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

##  Görev 2: CLTV Veri Yapısının Oluşturulması

### Adım 1: Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız.

In [15]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [16]:
today_date = dt.datetime(2021, 6, 1)

### Adım 2: customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı yeni bir cltv dataframe'i oluşturunuz.

* **recency_cltv_weekly** = Son satın alma üzerinden geçen haftalık zaman
* **T_weekly** = Analiz tarihinden ne kadar süre önce ilk satın alma yapmış. Müşterinin yaşı
* **frequency** = Tekrar eden toplam satın alma sayısı
* **monetary_cltv_avg**= Satın alma başına ortalama kazanç

In [17]:
cltv_df = pd.DataFrame()

In [18]:
cltv_df["customer_id"] = df["master_id"]

In [19]:
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"] - df["first_order_date"]).dt.days) / 7
cltv_df["recency_cltv_weekly"] = cltv_df["recency_cltv_weekly"].astype("int64")

**recency_cltv_weekly** = Son satın alma üzerinden geçen haftalık zaman

In [20]:
cltv_df["T_weekly"] = ((today_date - df["first_order_date"]).astype('timedelta64[ns]')) / 7
cltv_df["T_weekly"] = cltv_df["T_weekly"].dt.days

**T_weekly** = Analiz tarihinden ne kadar süre önce ilk satın alma yapmış. Müşterinin yaşı

In [21]:
cltv_df["frequency"] = df["order_num_total"].astype("int64")

**frequency** = Tekrar eden toplam satın alma sayısı

In [22]:
cltv_df["monetary_cltv_avg"] = df["customer_value_total"].astype("int64") / df["order_num_total"].astype("int64")
cltv_df["monetary_cltv_avg"] = cltv_df["monetary_cltv_avg"].astype("int64")

**monetary_cltv_avg** = Satın alma başına ortalama kazanç

In [23]:
cltv_df = cltv_df[cltv_df["recency_cltv_weekly"] > 0]

Analizi son günün üstünde aldığımız için **recency_cltv_weekly** değeri 0 olamaz. Her ihtimale karşı önlem alıyoruz.

In [24]:
cltv_df = cltv_df[cltv_df["T_weekly"] > 1]

İlk satın alma analiz tarihinden önce yapılmış olması lazım. Yine önlem alıyoruz.

In [25]:
cltv_df = cltv_df[cltv_df['recency_cltv_weekly'] < cltv_df['T_weekly']]

**Recency** değeri mantıken **T_weekly**'den küçük olmalı. Hatalı bir sonuç almamak için uyguluyoruz.


## Görev 3: BG/NBD, Gamma-Gamma Modellerinin Kurulması ve CLTV’nin Hesaplanması

### Adım 1: BG/NBD modelini fit ediniz.
* 3 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_3_month olarak cltv dataframe'ine ekleyiniz.
* 6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_6_month olarak cltv dataframe'ine ekleyiniz.

**BG/NBD** modelinde ise müşterinin yapmış olduğu ortalama harcamayla ilgilenmiyoruz. Müşteri bu zamana kadar ne kadar alışveriş yapmış, alışverişleri arasında ne kadar zaman geçmişle ilgileniriz. Bu değerlere uygun olarak müşterinin yapabileceği alışverişleri tahmin etmeye çalışırız. Aşağıda da 3 aylık ve 6 aylık tahminleri gösteriyorum.

In [26]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [27]:
bgf.fit(cltv_df["frequency"],
        cltv_df["recency_cltv_weekly"],
        cltv_df["T_weekly"])

c:\users\oe\appdata\local\programs\python\python39\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19566 subjects, a: 0.00, alpha: 76.61, b: 0.00, r: 3.67>

In [29]:
cltv_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                                cltv_df["frequency"],
                                                cltv_df["recency_cltv_weekly"],
                                                cltv_df["T_weekly"])

In [30]:
cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                                cltv_df["frequency"],
                                                cltv_df["recency_cltv_weekly"],
                                                cltv_df["T_weekly"])

### Adım 2: Gamma-Gamma modelini fit ediniz. Müşterilerin ortalama bırakacakları değeri tahminleyip exp_average_value olarak cltv dataframe'ine ekleyiniz.

In [31]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

In [32]:
ggf.fit(cltv_df["frequency"], cltv_df["monetary_cltv_avg"])

<lifetimes.GammaGammaFitter: fitted with 19566 subjects, p: 4.15, q: 0.47, v: 4.08>

In [33]:
cltv_df["expected_average_value"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                                                            cltv_df["monetary_cltv_avg"])

Bir müşterinin işlem başına ne kadar kar getirebileceğini tahmin etmek için kullandık. Frekans ve ortalama harcamasının dikkate alma amacımız bir kullanıcı bizde ne kadar sık alışveriş yaptıysa o kadar çok yapmaya devam edebileceği tahminini yürütmemize olanak sağlar. Aynı şekilde bu zamana kadar ortalama ne kadar para harcadıysa ilerleyen dönemlerde o şekilde harcama yapacağını tahmin edebiliriz.

### Adım 3: 6 aylık CLTV hesaplayınız ve cltv ismiyle dataframe'e ekleyiniz.
* Cltv değeri en yüksek 20 kişiyi gözlemleyiniz.

Müşteri Yaşam Boyu Değeri Tahmini, bir müşterinin işlem başına ne kadar kar getirebileceğini tahmin etmek için kullanılır. **Frekans** ve **ortalama harcamasını** dikkate alma amacımız bir kullanıcı bizde ne kadar sık alışveriş yaptıysa o kadar çok yapmaya devam edebileceği tahminini yürütmemize olanak sağlar. Ancak burda **recency** ve **T** değerlerini de dahil ettik. Bu da müşterinin yaşı ve son alışverişinden bu yana geçen süreyi de dahil eder. Bu da daha isabetli bir tahminde bulunmamıza katkı sağlar.

In [34]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency_cltv_weekly"],
                                   cltv_df["T_weekly"],
                                   cltv_df["monetary_cltv_avg"],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01)

In [35]:
cltv_df["cltv"] = cltv

In [36]:
cltv_df.sort_values(by="cltv", ascending=False).head(20)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,expected_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2,7,4,1401,1.101,2.203,1448.103,3346.849
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6,13,11,758,1.965,3.930,767.249,3163.713
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51,101,7,1106,0.721,1.442,1127.078,1705.662
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62,67,52,166,4.652,9.304,166.486,1625.233
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3,34,8,601,1.266,2.533,611.236,1624.453
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8,9,2,862,0.795,1.591,922.770,1540.055
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9,13,17,259,2.768,5.537,261.196,1517.402
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52,58,31,228,3.091,6.182,229.072,1485.831
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14,15,3,743,0.874,1.748,777.343,1426.043
15516,9083981a-f59e-11e9-841e-000d3a38a36f,63,83,4,1090,0.577,1.154,1126.881,1364.336


## Görev 4: CLTV Değerine Göre Segmentlerin Oluşturulması

### Adım 1: 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine ekleyiniz.

In [37]:
cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4 , labels=["D", "C", "B", "A"])

In [38]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency_cltv_weekly,19566.000,95.302,74.424,1.000,51.000,76.000,109.000,433.000
T_weekly,19566.000,114.744,74.626,2.000,74.000,93.000,119.000,437.000
frequency,19566.000,4.982,4.118,2.000,3.000,4.000,6.000,57.000
monetary_cltv_avg,19566.000,151.218,73.307,22.000,103.000,136.000,181.000,1401.000
exp_sales_3_month,19566.000,0.575,0.231,0.149,0.434,0.535,0.666,4.652
exp_sales_6_month,19566.000,1.151,0.461,0.299,0.868,1.070,1.332,9.304
expected_average_value,19566.000,158.282,76.748,24.395,107.907,142.082,189.396,1448.103
cltv,19566.000,193.349,131.625,11.886,111.338,164.128,238.255,3346.849


In [39]:
cltv_df["cltv"].info()

<class 'pandas.core.series.Series'>
Index: 19566 entries, 0 to 19944
Series name: cltv
Non-Null Count  Dtype  
--------------  -----  
19566 non-null  float64
dtypes: float64(1)
memory usage: 305.7 KB


In [40]:
cltv_df["cltv"] = cltv_df["cltv"].astype(float)

In [41]:
cltv_df.groupby("cltv_segment").agg({"cltv": ["sum", "count", "mean"]})

cltv              
                     sum count    mean
cltv_segment                          
D             389278.975  4892  79.575
C             670711.389  4891 137.132
B             968023.166  4891 197.919
A            1755055.782  4892 358.760

Segmentlerimiz eşit dağılmışlar. Ortalamalarında bir sıkıntı gözükmüyor. cltv sum değerleri arasında çok fark olmaması veriyi düzgün böldüğümüzü gösteriyor.

### Adım 2: 4 grup içerisinden seçeceğiniz 2 grup için yönetime kısa kısa 6 aylık aksiyon önerilerinde bulununuz.

In [42]:
cltv_df[cltv_df["cltv_segment"] == "D"].describe().T

,count,mean,std,min,25%,50%,75%,max
recency_cltv_weekly,4892.000,139.168,96.479,1.000,69.000,101.000,200.000,432.000
T_weekly,4892.000,162.548,96.025,11.000,91.000,118.000,224.000,436.000
frequency,4892.000,3.777,2.150,2.000,2.000,3.000,4.250,18.000
monetary_cltv_avg,4892.000,92.409,30.378,22.000,73.000,89.000,108.000,284.000
exp_sales_3_month,4892.000,0.408,0.123,0.149,0.325,0.411,0.483,1.051
exp_sales_6_month,4892.000,0.817,0.245,0.299,0.651,0.822,0.967,2.102
expected_average_value,4892.000,97.898,31.799,24.395,76.936,94.011,113.344,305.482
cltv,4892.000,79.575,21.575,11.886,64.490,83.022,97.784,111.334


D segmentindeki insanların alışveriş alışkanlığı yukarıdaki tablodaki gibidir. Bu tabloya göre D segmentindeki bir kişinin 6 aylık süreç içerisinde ortalama 0.817 alışveriş yapması bekleniyor. Bu değeri arttırmak için D segmentindeki ortalama harcamaları da dikkate alınarak özel indirimler uygulanabilir.

In [43]:
cltv_df[cltv_df["cltv_segment"] == "A"].describe().T

,count,mean,std,min,25%,50%,75%,max
recency_cltv_weekly,4892.000,67.869,51.572,1.000,35.000,62.000,86.000,433.000
T_weekly,4892.000,83.244,50.848,2.000,54.000,79.000,98.000,437.000
frequency,4892.000,6.666,6.044,2.000,3.000,5.000,8.000,57.000
monetary_cltv_avg,4892.000,227.856,88.720,64.000,170.000,209.000,263.000,1401.000
exp_sales_3_month,4892.000,0.770,0.297,0.212,0.574,0.710,0.882,4.652
exp_sales_6_month,4892.000,1.539,0.594,0.425,1.147,1.419,1.765,9.304
expected_average_value,4892.000,236.921,94.224,64.615,175.401,217.396,275.029,1448.103
cltv,4892.000,358.760,157.069,238.264,267.898,310.231,391.540,3346.849


A segmentindeki insanların alışveriş alışkanlığı yukarıdaki gibidir. Bu segmentteki kişiler sık şekilde alışveriş yapmaktadır.Bu segmentteki kişiler için ürün yelpazemizi genişletmek daha farklı modeller üretmek bu kişileri daha fazla alışveriş yapmaya itebilir.